# Complete Configuration File
 
```yaml
n_epochs: <NUMBER OF EPOCHS>  # Number of training epochs.
seed: 1                       # Seed for initialising random numbers
patience: None                # Number of epochs without improvement before trainings gets terminated.
n_models: 1                   # Number of models to be trained at once.
n_jitted_steps: 1             # Number of train batches to be processed in a compiled loop. 
                              # Can yield singificant speedups for small structures or small batch sizes.

data:
  directory: models/          # Path to the directory where the training results and checkpoints will be written.
  experiment: apax            # Name of  the model. Distinguishes it from the other models trained in the same `directory`.
  data_path: <PATH>           # Path to a single dataset file. Set either this or `val_data_path` and `train_data_path`.
  train_data_path: <PATH>     # Path to a training dataset. Set this and `val_data_path` if your data comes pre-split.
  val_data_path: <PATH>       # Path to a validation dataset. Set this and `train_data_path` if your data comes pre-split.
  test_data_path: <PATH>      # Path to a test dataset. Set this, `train_data_path` and `val_data_path` if your data comes pre-split.

  n_train: 1000               # Number of training datapoints from `data_path`.
  n_valid: 100                # Number of validation datapoints from `data_path`.

  batch_size: 32              # Number of training examples to be evaluated at once.
  valid_batch_size: 100       # Number of validation examples to be evaluated at once.

  shift_method: "per_element_regression_shift"
  shift_options:
    energy_regularisation: 1.0    # Magnitude of the regularization in the per-element energy regression.
  shuffle_buffer_size: 1000       # Size of the `tf.data` shuffle buffer.

  pos_unit: Ang
  energy_unit: eV

  additional_properties_info:     # Dict of property name, shape (ragged or fixed) pairs

model:
  n_basis: 7                  # Number of uncontracted gaussian basis functions.
  n_radial: 5                 # Number of contracted basis functions.
  nn: [512, 512]              # Number of hidden layers and units in those layers.

  r_max: 6.0                  # Position of the first uncontracted basis function's mean.
  r_min: 0.5                  # Cutoff radius of the descriptor.

  use_zbl: false              # 

  b_init: normal              # Initialization scheme for the neural network biases. Either `normal` or `zeros`.
  descriptor_dtype: fp64
  readout_dtype: fp32
  scale_shift_dtype: fp32

loss:
- loss_type: structures       # Weighting scheme for atomic contributions.
                              # See the MLIP package for reference 10.1088/2632-2153/abc9fe for details
  name: energy                # Keyword of the quantity e.g `energy`.
  weight: 1.0                 # Weighting factor in the overall loss function.
- loss_type: structures
  name: forces
  weight: 4.0

metrics:
- name: energy                # Keyword of the quantity e.g `energy`.
  reductions:                 # List of reductions performed on the difference between target and predictions.
                              # Can be mae, mse, rmse for energies and forces. For forces it is also possible to use `angle`.
  - mae
- name: forces
  reductions:
  - mae
  - mse

optimizer:
  opt_name: adam            # Name of the optimizer. Can be any `optax` optimizer.
  opt_kwargs: {}            # Optimizer keyword arguments. Passed to the `optax` optimizer.
  emb_lr: 0.03              # Learning rate of the elemental embedding contraction coefficients.
  nn_lr: 0.03               # Learning rate of the neural network parameters.
  scale_lr: 0.001           # Learning rate of the elemental output scaling factors.
  shift_lr: 0.05            # Learning rate of the elemental output shifts.
  zbl_lr: 0.001             # 
  transition_begin: 0       # Number of training steps (not epochs) before the start of the linear learning rate schedule.

callbacks:
- name: csv                 # Keyword of the callback used. Currently we implement "csv" and "tensorboard".

progress_bar:
  disable_epoch_pbar: false   # Set to True to disable the epoch progress bar.
  disable_nl_pbar: false      # Set to True to disable the NL precomputation progress bar.


checkpoints:
  ckpt_interval: 1                # Number of epochs between checkpoints.
  
                                  # The options below are used for transfer learning
  base_model_checkpoint: null     # Path to the folder containing a pre-trained model ckpt.
  reset_layers: []                # List of layer names for which the parameters will be reinitialized.

```